In [186]:
import numpy as np
import pandas as pd
from scipy.optimize import fsolve
import matplotlib.pyplot as plt

In [187]:
def presion_vapor_sat(T):
    if (T>=0):
        es = 611 * np.exp(17.27*T/(T + 237.3))
    else:
        es = 611 * np.exp(21.87*T/(T + 265.5))
    return es 

def rel_mezcla(e,p):
    return 0.622 * e / p

def presion_adiabatica(T, T0, P0, alpha=9.77/1000, g=9.81, R = 287):
    P = P0 * ((T + 273.15)/(T0 + 273.15)) ** (g/(alpha * R))
    return P

def latente(T):
    return 2.501 - 0.00236 * T

def humedad_absoluta(e, T, Ra = 287):
    rho_v = 0.622 * e / (Ra * (T + 273.15))
    return rho_v

# Problema 6

A continuación se determina el nivel de saturación:

In [188]:
T0 = 10 # Temperatura en la superficie (C)
RH = 70 # Humedad relativa (%)
zt = 1 # Altura máxima (km)
dz = 0.03 # Paso de elevación (km)
cp = 1005 # Calor específico a presión constante (J/kg K)

p0 = 10 ** 5 # Presión en la superficie (Pa)
alpha = 9.77 # Tasa de lapso adiabática (C/km)
Ra = 287 # Constante de gas de aire húmedo (J/kg K)
g = 9.81 # Gravedad (m/s2)

e0 = RH * presion_vapor_sat(T0) / 100 # Presión de vapor en la superficie
r0 = rel_mezcla(e0, p0) # Relación de mezcla (kg/kg)

def func(x, T0, P0, r0, alpha=9.77/1000, g=9.81, Ra=287):
    es = presion_vapor_sat(x)
    ps = presion_adiabatica(x, T0, P0, alpha, g, Ra)
    rs = rel_mezcla(es, ps)
    return r0 - rs

Tc = fsolve(func, x0 = T0, args=(T0, p0, r0))[0]
pc = presion_adiabatica(Tc, T0, p0)
es = presion_vapor_sat(Tc)

zc = (T0 - Tc) / alpha

print(f'La relación de mezcla en la superficie es {r0 * 1000:3.2f} g/kg')
print(f'La temperatura de saturación es {Tc:3.2f} C y la presión de saturación es {pc:3.2f} Pa')
print(f'La presión de vapor de saturación en el nivel de saturación es {es:3.2f} Pa')
print(f'El nivel de saturación es {zc:3.3f} km')

La relación de mezcla en la superficie es 5.35 g/kg
La temperatura de saturación es 3.66 C y la presión de saturación es 92385.47 Pa
La presión de vapor de saturación en el nivel de saturación es 794.38 Pa
El nivel de saturación es 0.649 km


Ahora se determina la temperatura potencial equivalente para hacer el cálculo de los perfiles de temperatura, presión y relación de mezcla hasta el techo de la montaña:

In [189]:
lh = latente(Tc)
theta_e = (T0 + 273.15) * np.exp(lh * 10**6 * r0 / (cp * (Tc + 273.15)))

print(f'La temperatura potencial equivalente es {theta_e:3.2f} K')

La temperatura potencial equivalente es 297.05 K


Ahora se calcula la relación de mezcla cada 30m

In [193]:
def func(x, T, theta_e, P0, alpha=9.77/1000, g=9.81, Ra=287, cp = 1005):
    lh = latente(T)
    es = presion_vapor_sat(T)
    rs = rel_mezcla(es, x)
    z = theta_e - (T + 273.15)* (P0 / x) ** (Ra * alpha / g) * np.exp(lh * 10**6 * rs / (cp * (T + 273.15)))
    return z

z = np.arange(zc, zt + dz, dz); z[-1] = zt
alpha_s = 5 # tasa de lapso de saturación (C/km)
T = P = theta = rs = []

for z_i in z:
    T_i = Tc - alpha_s * (z_i - zc)
    P_i = fsolve(func, x0 = pc, args = (T_i, theta_e, p0))[0]
    es_i = presion_vapor_sat(T_i)
    rs_i = rel_mezcla(es_i, P_i)
    rho_v_i = humedad_absoluta(es_i, T_i)
    theta_i = (T_i + 273.15) * (p0/P_i) ** (alpha/1000 * Ra / g)
    
    T = T + [T_i]
    P = P + [P_i]
    rs = rs + [rs_i]
    theta = theta + [theta_i]

# Cálculo de diferencia de relaciones de mezcla de saturación cada 30 m
drs = -np.diff(rs) * 1000
out = pd.DataFrame(dict(z_km=z[1:], dr_mm = drs))
out

,z_km,dr_mm
0,0.67871,0.040230
1,0.70871,0.040018
2,0.73871,0.039806
3,0.76871,0.039595
4,0.79871,0.039384
5,0.82871,0.039174
6,0.85871,0.038964
7,0.88871,0.038754
8,0.91871,0.038545
9,0.94871,0.038336


Cuando la masa de aire regresa a la superficie, esta adquiere la temperatura potencial calculada al nivel del techo de la montaña. A contunuación, se calcula la nueva humedad relativa de la masa de aire:

In [196]:
rs_t = rs[-1] # relación de mezcla en el techo de la montaña (kg/kg)
theta_t = theta[-1] # temperatura potencial en el techo de la montaña (K)
t0_nuevo = theta_t - 273.15 # Nueva temperatura de la masa de aire a nivel de superficie
es_nuevo = presion_vapor_sat(t0_nuevo) # Nueva presión de vapor de saturación a nivel de superficie (Pa)
e_nuevo = p0 * rs_t / 0.622 # Nueva presión de vapor a nivel de superficie (Pa)
RH_nuevo = e_nuevo/es_nuevo * 100 # Nueva humedad relativa a nivel de superficie (%)

print(f'Los nuevos valores a nivel de superficie para presión de vapor (normal y saturada) son {e_nuevo:3.2f} Pa y {es_nuevo:3.2f} Pa, respectivamente')
print(f'Los nuevos valores a nivel de superficie para temperatura del aire y humedad relativa son {t0_nuevo:3.2f} C y {RH_nuevo:3.2f} %, respectivamente')

Los nuevos valores a nivel de superficie para presión de vapor (normal y saturada) son 786.24 Pa y 1318.70 Pa, respectivamente
Los nuevos valores a nivel de superficie para temperatura del aire y humedad relativa son 11.06 C y 59.62 %, respectivamente


# Problema 7

Este problema es similar en metodología al problema anterior. No obstante, en lugar de trabajar con las relaciones de mezcla en la superficie y en el nivel de condensación, se usará la relación de humedades relativas entre estos dos niveles.

In [220]:
RH = 90
T0 = 25
alpha = 6

def func(x, RH, T0, alpha, g=9.81, R=287):
    es_c = presion_vapor_sat(T0 - alpha * x)
    es0 = presion_vapor_sat(T0)
    return 100/RH - ((T0 - alpha * x + 273.15)/(T0 + 273.15))**(g/(R*alpha/1000))*(es0/es_c)

zc = fsolve(func, x0 = 10, args=(RH, T0, alpha))[0]
print(f'El nivel de condensación es {zc:3.3f} km')

El nivel de condensación es 0.429 km


# Problema 9

In [181]:
Tavg = 20
Pavg = 85000
v = 2
H = 500
cp = 1005
p0 = 10**5
dz = 100
R = 287
rho_w = 1000

alpha_s = 5
Tc = Tavg + alpha_s * 250/1000
Tt = Tc - alpha_s*H/1000

def fun2(x, T, theta_e, P0, cp = 1005):
    lh = latente(T)
    es = presion_vapor_sat(T)
    rs = rel_mezcla(es, x)
    z = theta_e - (T + 273.15)* (P0 / x) ** 0.286 * np.exp(lh * 10**6 * rs / (cp * (T + 273.15)))
    return z
    
def fun0(x, Tc, p0, H, dz, cp=1005):
    
    es = presion_vapor_sat(Tc)
    r0 = rel_mezcla(es, x)
    lh = latente(Tc)
    theta_e = (Tc + 273.15) * (p0 / x) ** 0.286 * np.exp(lh * 10 ** 6 * r0 / (cp * (Tc + 273.15)))
    
    z = np.arange(0, H + dz, dz)
    P = []
    for z_i in z:
        T_i = Tc - alpha_s/1000 * z_i
        P_i = fsolve(fun2, x0 = p0, args=(T_i, theta_e, p0))
        P = P + [P_i]
    
    dP = 0
    for i, P_i in enumerate(P):
        if  i > 0:
            dP = dP + (P_i + P[i-1])/2 * dz
    
    z = Pavg * H - dP
    return z
    
Pc = fsolve(fun0, x0 = p0, args=(Tc, p0, H, dz, cp))[0]
es = presion_vapor_sat(Tc)
r0 = rel_mezcla(es, Pc)
theta_e = (Tc + 273.15) * (p0 / Pc) ** 0.286 * np.exp(lh * 10 ** 6 * r0 / (cp * (Tc + 273.15)))

Pt = fsolve(fun2, x0=Pc, args=(Tt, theta_e, p0))[0]
es1 = presion_vapor_sat(Tt)
r1 = rel_mezcla(es1, Pt)

dr = r0 - r1
rho_a = Pc/(R * (Tc + 273.15))
i = dr * rho_a * v / rho_w * (1000 * 3600)

print(f'La intensidad máxima es {i:3.2f} mm/hr')

La intensidad máxima es 10.43 mm/hr
